In [1]:
import os
import numpy as np # to process the data
import random
import math
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
class FlowerDataset:
    def __init__(self, directory):
        self.directory = directory
        self.images = []
        self.labels = []

    def load_data(self):
        for i, img in enumerate(os.listdir(self.directory)):
            if i >= 10:  # only load the first 10 images
                break
            try:
                img_path = os.path.join(self.directory, img)
                with Image.open(img_path) as img_array:  # read the image
                    gray_array = img_array.convert('L')  # ensure image is grayscale
                    resized_array = gray_array.resize((150, 150))  # resize the image
                    self.images.append(list(resized_array.getdata()))  # keep image in 1D
                    label = img.split('_')[0]  # extract label from filename
                    self.labels.append(label)
            except Exception as e:
                print(f"Error loading image {img}: {e}")

    def preprocess_data(self):
        # Normalize pixel values
        self.images = [[pixel / 255.0 for pixel in image] for image in self.images]
        print(f"Loaded {len(self.images)} images")

# Load and preprocess the data
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()

# Print the shape of the preprocessed images
print(f"Shape of the preprocessed images: {len(dataset.images)} images, each with {len(dataset.images[0])} pixels")

# Dimensions of the first preprocessed image
print(f"Dimensions of the first preprocessed image: {len(dataset.images[0])} pixels")

# Split the data into training, validation, and test sets
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# Custom function to split the dataset
def custom_train_test_split(data, labels, train_size):
    train_count = int(len(data) * train_size)
    return data[:train_count], data[train_count:], labels[:train_count], labels[train_count:]

# train is now 75% of the entire data set
x_train, x_temp, y_train, y_temp = custom_train_test_split(dataset.images, dataset.labels, train_ratio)

# test is now 10% of the initial data set, validation is now 15%
val_count = int(len(x_temp) * (validation_ratio / (test_ratio + validation_ratio)))
x_val, x_test = x_temp[:val_count], x_temp[val_count:]
y_val, y_test = y_temp[:val_count], y_temp[val_count:]

# Print the sizes of the datasets
print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

# Print the unique classes in the training set
unique_classes = set(y_train)
print(f"Unique classes in the training set: {unique_classes}")

Loaded 10 images
Shape of the preprocessed images: 10 images, each with 22500 pixels
Dimensions of the first preprocessed image: 22500 pixels
Train set size: 7, Validation set size: 1, Test set size: 2
Unique classes in the training set: {'orchids', 'daisies', 'peonies', 'hibiscus', 'lilies', 'hydrangeas', 'tulip'}


In [33]:
def relu(x):
    if isinstance(x, list):
        return [max(0, xi) for xi in x]
    else:
        return max(0, x)

def relu_derivative(x):
    if isinstance(x, list):
        return [1 if xi > 0 else 0 for xi in x]
    else:
        return 1 if x > 0 else 0

def mean_squared_error(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)

def gradient_mse(y_true, y_pred):
    return [2 * (yp - yt) for yt, yp in zip(y_true, y_pred)]

def softmax(x):
    exps = [math.exp(i) for i in x]
    sum_of_exps = sum(exps)
    return [j / sum_of_exps for j in exps]

def cross_entropy(predictions, targets):
    # Assuming predictions is a list of probabilities and targets is a list of one-hot encoded classes
    N = len(predictions)
    ce = 0
    for p, t in zip(predictions, targets):
        ce -= t * math.log(p) if p > 0 else 0  # Adding check to prevent math domain error
    ce /= N
    return ce

def dot_product(x, y):
    return sum(xi * yi for xi, yi in zip(x, y))

def matrix_multiply(X, Y):
    return [[dot_product(X_row, Y_col) for Y_col in zip(*Y)] for X_row in X]

def transpose(matrix):
    return list(map(list, zip(*matrix)))

def matrix_addition(X, Y):
    return [[xi + yi for xi, yi in zip(x, y)] for x, y in zip(X, Y)]

def scalar_multiply(matrix, scalar):
    return [[scalar * element for element in row] for row in matrix]

def reshape_input(flat_input, image_height, image_width):
    return [flat_input[i * image_width:(i + 1) * image_width] for i in range(image_height)]
    
# Layer classes
class Layer:
    def forward(self, input):
        raise NotImplementedError

    def backward(self, input, gradient):
        raise NotImplementedError

class Conv2D(Layer):
    def __init__(self, num_filters, filter_size, image_height, image_width):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.image_height = image_height
        self.image_width = image_width
        # Initialize filters with small random values
        self.filters = [[random.uniform(-0.1, 0.1) for _ in range(filter_size * filter_size)] for _ in range(num_filters)]

    def forward(self, input):
        # Calculate the padding size
        padding = self.filter_size // 2

        # Reshape 1D input into 2D and pad with zeros
        input_2d = [input[i * self.image_width:(i + 1) * self.image_width] for i in range(self.image_height)]
        padded_input = [[0] * (self.image_width + 2 * padding) for _ in range(padding)] + \
                       [[0] * padding + row + [0] * padding for row in input_2d] + \
                       [[0] * (self.image_width + 2 * padding) for _ in range(padding)]
        
        # Adjusted output dimensions after padding
        output_height = len(padded_input) - self.filter_size + 1
        output_width = len(padded_input[0]) - self.filter_size + 1
        output = [[0 for _ in range(output_width)] for _ in range(output_height)]

        print(padded_input[9])
        
        # Perform the convolution operation
        for f in range(self.num_filters):
            for i in range(output_height):
                for j in range(output_width):
                    print(i+j)
                    
                    region = [padded_input[i+x][j+y] for x in range(self.filter_size) for y in range(self.filter_size)]
                    output[i][j] = sum(r * k for r, k in zip(region, self.filters[f]))

                    print(f"Filter {f}, Position ({i}, {j}):")
                    print("Region:", region)
                    print("Filter:", self.filters[f])
                    print("Output Value:", output[i][j])
        return output

    def backward(self, input, output_gradient, learning_rate):
        # Initialize gradients for filters with zeros
        d_filters = [[[0 for _ in range(self.filter_size * self.filter_size)] for _ in range(self.filter_size)] for _ in range(self.num_filters)]
        
        # Initialize input_gradient with zeros
        input_gradient = [[0 for _ in range(self.image_width)] for _ in range(self.image_height)]

        # Loop over the output gradient and filters to accumulate the input_gradient
        for f in range(self.num_filters):
            for i in range(len(output_gradient)):
                for j in range(len(output_gradient[0])):
                    # Determine the region of the input that contributed to the output_gradient[i][j]
                    for x in range(self.filter_size):
                        for y in range(self.filter_size):
                            # Calculate the coordinates in the input
                            in_x = i + x
                            in_y = j + y
                            # Accumulate the gradient for the input
                            if 0 <= in_x < self.image_height and 0 <= in_y < self.image_width:
                                input_gradient[in_x][in_y] += output_gradient[i][j] * self.filters[f][x * self.filter_size + y]
                            # Accumulate the gradient for the filter
                            d_filters[f][x * self.filter_size + y] += input_2d[in_x][in_y] * output_gradient[i][j]

        # Update filters with the calculated gradients
        for f in range(self.num_filters):
            for k in range(self.filter_size * self.filter_size):
                self.filters[f][k] -= learning_rate * d_filters[f][k]

        return input_gradient

class MaxPooling2D(Layer):
    def __init__(self, pool_size, image_height, image_width):
        self.pool_size = pool_size
        self.image_height = image_height
        self.image_width = image_width

    def forward(self, input):
        # Reshape 1D input into 2D
        input = reshape_input(input, self.image_height, self.image_width)
        # Rest of the code remains the same...

class Flatten(Layer):
    def forward(self, input):
        self.input_shape = (len(input), len(input[0]))  # save this for backward pass
        return [item for sublist in input for item in sublist]  # flatten the list

    def backward(self, input, gradient):
        return [gradient[i:i+self.input_shape[1]] for i in range(0, len(gradient), self.input_shape[1])]

class Dense(Layer):
    def __init__(self, num_inputs, num_outputs):
        # Initialize weights and biases
        self.weights = [[random.random() for _ in range(num_inputs)] for _ in range(num_outputs)]
        self.biases = [random.random() for _ in range(num_outputs)]

    def forward(self, input):
        # Compute the weighted sum of inputs plus biases
        return [dot_product(weights_row, input) + b for weights_row, b in zip(self.weights, self.biases)]

    def backward(self, input, gradient):
        # Compute gradients with respect to weights and biases
        d_weights = scalar_multiply(transpose([gradient]), [input])
        d_biases = gradient
        d_input = dot_product(self.weights, gradient)
        return d_input, d_weights, d_biases

class Dropout(Layer):
    def __init__(self, rate):
        self.rate = rate

    def forward(self, input):
        self.input_shape = (len(input), len(input[0]))  # save this for backward pass
        self.mask = [[random.random() > self.rate for _ in range(self.input_shape[1])] for _ in range(self.input_shape[0])]
        return [[input[i][j] * self.mask[i][j] for j in range(self.input_shape[1])] for i in range(self.input_shape[0])]

    def backward(self, input, gradient):
        return [[gradient[i][j] * self.mask[i][j] for j in range(self.input_shape[1])] for i in range(self.input_shape[0])]

# Activation functions as layers
class ReLU(Layer):
    def forward(self, input):
        output = [relu(x) for x in input]
        return output

    def backward(self, input, gradient):
        # Apply the derivative of ReLU to the gradient
        return [g * relu_derivative(x) for x, g in zip(input, gradient)]

# Loss function as a class
class SoftmaxCrossEntropyLoss:
    def forward(self, logits, labels):
        self.predictions = softmax(logits)
        return cross_entropy(self.predictions, labels)

    def backward(self, logits, labels):
        return [p - l for p, l in zip(self.predictions, labels)]

In [ ]:
class CustomCNNModel:
    def __init__(self, num_epochs, learning_rate, threshold, early_stopping_rounds):
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.early_stopping_rounds = early_stopping_rounds
        self.layers = [
            Conv2D(num_filters=32, filter_size=3, image_height=150, image_width=150),
            ReLU(),
            MaxPooling2D(pool_size=2, image_height=148, image_width=148),  # image size is reduced by filter_size - 1
            Flatten(),
            Dense(num_inputs=74*74*32, num_outputs=10),  # 74 is the new image size after pooling, 32 is the number of filters, 10 is the number of classes
            Dropout(rate=0.5)
        ]
        self.loss = SoftmaxCrossEntropyLoss()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def compute_loss_and_gradients(self, x, y):
        logits = self.forward(x)
        loss = self.loss.forward(logits, y)
        gradient = self.loss.backward(logits, y)
        for layer in reversed(self.layers):
            gradient = layer.backward(x, gradient)
        return loss

    def update_weights(self):
        for layer in self.layers:
            if isinstance(layer, Dense) or isinstance(layer, Conv2D):
                for i, (w, g) in enumerate(zip(layer.weights, layer.d_weights)):
                    for j, _ in enumerate(w):
                        layer.weights[i][j] -= self.learning_rate * g[j]
                for i, b in enumerate(layer.biases):
                    layer.biases[i] -= self.learning_rate * layer.d_biases[i]

    def train(self, x_train, y_train, x_val, y_val):
        history = []
        best_val_loss = float('inf')
        no_improvement_rounds = 0

        for epoch in range(self.num_epochs):
            loss = self.compute_loss_and_gradients(x_train, y_train)
            self.update_weights()
            val_loss = self.compute_loss_and_gradients(x_val, y_val)

            if val_loss < best_val_loss - self.threshold:
                best_val_loss = val_loss
                no_improvement_rounds = 0
            else:
                no_improvement_rounds += 1

            if no_improvement_rounds >= self.early_stopping_rounds:
                print(f"Early stopping on epoch {epoch}")
                break

            history.append((loss, val_loss))
            print(f"Epoch {epoch}: loss = {loss}, val_loss = {val_loss}")

        return history



model = CustomCNNModel(num_epochs=10, learning_rate=0.01, threshold=0.01, early_stopping_rounds=5)
history = model.train(x_train, y_train, x_val, y_val)